In [117]:
import pandas as pd
import numpy as np
import calendar
import datetime
import json

spire_source = pd.read_csv("C:\\Users\\MarkR\\Documents\\UTwente\\GFW model\\Git clone scoring\\vessel-scoring\\datasets\\SouthChinaSea_AllSources_0601_1501_2022.csv", header=0)

spire_source.head()

C:\Users\MarkR\AppData\Local\Temp\ipykernel_6936\351253157.py:7: DtypeWarning: Columns (11,12,13,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  spire_source = pd.read_csv("C:\\Users\\MarkR\\Documents\\UTwente\\GFW model\\Git clone scoring\\vessel-scoring\\datasets\\SouthChinaSea_AllSources_0601_1501_2022.csv", header=0)


,created_at,timestamp,mmsi,msg_type,latitude,longitude,speed,course,heading,rot,...,eta,destination,status,maneuver,accuracy,to_bow,to_stern,to_port,to_starboard,collection_type
0,2022-01-08 08:41:13.615472 UTC,2022-01-08 05:45:38 UTC,667001585,3,7.382017,115.643417,8.8,69.5,79.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,satellite
1,2022-01-14 14:57:16.33786 UTC,2022-01-14 14:57:14 UTC,310772000,3,15.009760,119.638408,16.1,350.7,350.0,3.0,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,terrestrial
2,2022-01-14 06:07:04.177057 UTC,2022-01-14 04:43:06 UTC,305755000,3,2.549142,105.061157,12.4,206.9,210.0,-12.0,...,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,satellite
3,2022-01-07 06:30:17.442102 UTC,2022-01-07 06:30:16 UTC,211256150,3,14.234838,119.625160,1.8,235.0,166.0,5.0,...,NaN,NaN,3.0,0.0,0.0,NaN,NaN,NaN,NaN,terrestrial
4,2022-01-06 07:06:21.741443 UTC,2022-01-06 06:42:51 UTC,667001585,3,7.084300,108.917217,8.1,90.6,80.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,satellite


In [75]:
# Would like to select all ship_and_cargo_type == 30 since this displays all fishing vessels, however, all these rows contain NaN values in the other columns.
spire_source2 = spire_source #[spire_source['ship_and_cargo_type'] == 30]
spire_source3 = spire_source2[~spire_source2['latitude'].isnull()]

In [76]:
spire_lim = spire_source3[['timestamp', 'mmsi', 'latitude', 'longitude', 'course', 'speed']]
spire_lim.head()


,timestamp,mmsi,latitude,longitude,course,speed
0,2022-01-08 05:45:38 UTC,667001585,7.382017,115.643417,69.5,8.8
1,2022-01-14 14:57:14 UTC,310772000,15.009760,119.638408,350.7,16.1
2,2022-01-14 04:43:06 UTC,305755000,2.549142,105.061157,206.9,12.4
3,2022-01-07 06:30:16 UTC,211256150,14.234838,119.625160,235.0,1.8
4,2022-01-06 06:42:51 UTC,667001585,7.084300,108.917217,90.6,8.1


In [110]:
spire_list = spire_lim.groupby('mmsi', as_index=False).size()

# spire_list = spire_list[~spire_list['mmsi'].isin([200000000, 663000000, 416004981])]

max_mmsi = spire_list[spire_list['size'] == max(spire_list['size'])]

spire_max_mmsi = spire_lim[spire_lim['mmsi'] == max_mmsi['mmsi'].iloc[0]]

spire_max_mmsi = spire_max_mmsi.sort_values(by=['timestamp'])

spire_max_mmsi

,timestamp,mmsi,latitude,longitude,course,speed
1173978,2022-01-06 00:01:45.02 UTC,636011491,1.573817,104.815150,144.0,0.2
1175221,2022-01-06 00:09:36.14 UTC,636011491,1.573900,104.815267,9.0,0.1
1175146,2022-01-06 00:18:14.53 UTC,636011491,1.573867,104.815100,282.0,0.1
1175896,2022-01-06 00:24:11.2 UTC,636011491,1.573833,104.815100,290.0,0.0
1175378,2022-01-06 00:24:36.49 UTC,636011491,1.573833,104.815083,290.0,0.0
...,...,...,...,...,...,...
1173309,2022-01-15 23:21:27.47 UTC,636011491,1.572117,104.814050,325.0,0.1
1174957,2022-01-15 23:33:24.01 UTC,636011491,1.572083,104.813933,354.0,0.1
1175867,2022-01-15 23:36:29.34 UTC,636011491,1.572098,104.813917,30.0,0.1
1174660,2022-01-15 23:42:30.18 UTC,636011491,1.572067,104.813950,21.0,0.0


In [111]:
spire_final = spire_max_mmsi[['timestamp', 'course', 'speed', 'latitude', 'longitude']]
spire_final = spire_final.rename(columns={'latitude': 'lat', 'longitude': 'lon'})
spire_final['timestamp'] = pd.to_datetime(spire_final['timestamp'])
spire_final['timestamp'] = spire_final['timestamp'].astype('int64')//1e9

spire_final

,timestamp,course,speed,lat,lon
1173978,1.641427e+09,144.0,0.2,1.573817,104.815150
1175221,1.641428e+09,9.0,0.1,1.573900,104.815267
1175146,1.641428e+09,282.0,0.1,1.573867,104.815100
1175896,1.641429e+09,290.0,0.0,1.573833,104.815100
1175378,1.641429e+09,290.0,0.0,1.573833,104.815083
...,...,...,...,...,...
1173309,1.642289e+09,325.0,0.1,1.572117,104.814050
1174957,1.642290e+09,354.0,0.1,1.572083,104.813933
1175867,1.642290e+09,30.0,0.1,1.572098,104.813917
1174660,1.642290e+09,21.0,0.0,1.572067,104.813950


In [119]:
spire_final.to_json("C:\\Users\\MarkR\\Documents\\UTwente\\GFW model\\Git clone scoring\\vessel-scoring\\datasets\\Spire_test.json", orient='records')

# dictionary_varibale = [ 
#     dict([
#         (colname, row[i]) 
#         for i,colname in enumerate(spire_final.columns)
#     ])
#     for row in spire_final.values
# ]
# print(json.dumps(dictionary_varibale))

In [ ]:
f = open("C:\\Users\\MarkR\\Documents\\UTwente\\GFW model\\Git clone scoring\\vessel-scoring\\datasets\\Spire_test.json")

counter = 0
for row in json.load(f):
    row = {key: float(value) for (key, value) in row.items()
                if key in ('timestamp', 'course', 'speed', 'lat', 'lon')
                  and value is not None}
    row['timestamp'] = datetime.datetime.fromtimestamp(int(row['timestamp']))
    print(row)